# Imports

In [100]:
import numpy as np
from scipy import ndimage

from typing import Tuple

# Utils

In [201]:
def analyze_clusters(microplate: np.ndarray, min_size: int = 2) -> Tuple:
    """
    Analyze cell clusters on a microplate 
    
    Parameters
    ----------
    
    Returns
    -------
    tuple
        (number of positive cells, number of clusters, size of largest cluster)
    """
    matrix = np.nan_to_num(microplate, 0)
    structure = ndimage.generate_binary_structure(2, 1)
    
    clusters, num_clusters = ndimage.label(matrix, structure)
    cluster_sizes = ndimage.sum(matrix, clusters, range(0, num_clusters + 1))
    
    mask = (cluster_sizes >= min_size)[clusters]
    clusters = clusters[mask]
    
    unique_labels, label_counts = np.unique(clusters, return_counts=True)

    return int(np.sum(matrix)), len(unique_labels), label_counts.max()

# Settings

In [202]:
num_microplates = 10000 # the number of microplates to simulate    
shape = (8, 12)         # shape of each microplate, `(nrows, ncols)`
prevalence = 0.4        # population prevalence, binomial success probability 
num_controls = 6        # the number of control wells per microplate
seed = 123              # passed to np.random.seed

# Generate samples

In [203]:
np.random.seed(123)

In [204]:
num_samples = num_microplates * (shape[0] * shape[1] - num_controls)
samples = np.random.binomial(1, prevalence, size=num_samples)
samples.shape

(900000,)

In [205]:
microplates = np.split(samples, num_microplates)
len(microplates)

10000

In [206]:
microplate = np.pad(microplates[0].astype(float), (num_controls,0), constant_values=np.nan)
microplate = np.reshape(microplate, shape)
microplate

array([[nan, nan, nan, nan, nan, nan,  1.,  0.,  0.,  0.,  1.,  0.],
       [ 1.,  1.,  0.,  0.,  0.,  1.,  0.,  0.,  0.,  1.,  0.,  0.],
       [ 0.,  0.,  1.,  1.,  1.,  1.,  1.,  0.,  0.,  0.,  0.,  1.],
       [ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  1.,  1.,  0.,  1.,  0.],
       [ 0.,  0.,  1.,  0.,  0.,  1.,  0.,  1.,  0.,  1.,  1.,  0.],
       [ 0.,  0.,  0.,  1.,  1.,  0.,  1.,  0.,  1.,  1.,  1.,  0.],
       [ 1.,  0.,  0.,  0.,  0.,  1.,  1.,  1.,  0.,  0.,  0.,  0.],
       [ 0.,  1.,  0.,  1.,  0.,  0.,  1.,  1.,  0.,  0.,  0.,  0.]])

In [207]:
analyze_clusters(microplate)

(34, 6, 6)

# Identify cell clusters

In [138]:
structure = ndimage.generate_binary_structure(2, 1)
structure

array([[False,  True, False],
       [ True,  True,  True],
       [False,  True, False]])

In [175]:
clusters, num_clusters = ndimage.label(np.nan_to_num(x, 0), structure)
clusters, num_clusters

(array([[ 0,  0,  0,  0,  0,  0,  1,  0,  0,  0,  2,  0],
        [ 3,  3,  0,  0,  0,  4,  0,  0,  0,  5,  0,  0],
        [ 0,  0,  4,  4,  4,  4,  4,  0,  0,  0,  0,  6],
        [ 0,  0,  0,  0,  0,  0,  0,  7,  7,  0,  8,  0],
        [ 0,  0,  9,  0,  0, 10,  0,  7,  0,  8,  8,  0],
        [ 0,  0,  0, 11, 11,  0, 12,  0,  8,  8,  8,  0],
        [13,  0,  0,  0,  0, 12, 12, 12,  0,  0,  0,  0],
        [ 0, 14,  0, 15,  0,  0, 12, 12,  0,  0,  0,  0]], dtype=int32),
 15)

In [176]:
sizes = ndimage.sum(x, clusters, range(0, num_clusters + 1))
sizes

array([0., 1., 1., 2., 6., 1., 1., 3., 6., 1., 1., 2., 6., 1., 1., 1.])

In [196]:
mask = (sizes > 1)[clusters]
mask

array([[False, False, False, False, False, False, False, False, False,
        False, False, False],
       [ True,  True, False, False, False,  True, False, False, False,
        False, False, False],
       [False, False,  True,  True,  True,  True,  True, False, False,
        False, False, False],
       [False, False, False, False, False, False, False,  True,  True,
        False,  True, False],
       [False, False, False, False, False, False, False,  True, False,
         True,  True, False],
       [False, False, False,  True,  True, False,  True, False,  True,
         True,  True, False],
       [False, False, False, False, False,  True,  True,  True, False,
        False, False, False],
       [False, False, False, False, False, False,  True,  True, False,
        False, False, False]])

In [197]:
labels = clusters[mask]
labels

array([ 3,  3,  4,  4,  4,  4,  4,  4,  7,  7,  8,  7,  8,  8, 11, 11, 12,
        8,  8,  8, 12, 12, 12, 12, 12], dtype=int32)

In [188]:
cluster_labels, cluster_sizes = np.unique(labels, return_counts=True)

In [190]:
num_positive = np.nansum(x).astype(int)

In [191]:
num_positive, len(cluster_labels), cluster_sizes.max()

(34, 6, 6)